In [23]:
import numpy as np
import torch
from torchvision.datasets import MNIST
from torchvision import transforms
from torch.utils.data import random_split, DataLoader
import torch.nn as nn
import torchsummary
import math

In [24]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [25]:
device

device(type='cuda')

In [26]:
transform = transforms.Compose([
    transforms.RandomAffine(degrees=15, translate=(0.5, 0.5), scale=(0.7, 1.3)),
    transforms.ToTensor()
])
# transform = transforms.ToTensor()

train_set = MNIST('./data', train=True, download=True, transform=transform)
testing = MNIST('./data', train=False, download=True, transform=transforms.ToTensor())

test_set, dev_set = random_split(testing, [5000, 5000])

In [27]:
len(train_set), len(test_set), len(dev_set)

(60000, 5000, 5000)

In [28]:
train_loader = DataLoader(train_set, batch_size=32, shuffle=True, pin_memory=True, num_workers=2)
dev_loader = DataLoader(dev_set, batch_size=1024, shuffle=False, pin_memory=True, num_workers=2)
test_loader = DataLoader(test_set, batch_size=1024, shuffle=False, pin_memory=True, num_workers=2)

In [29]:
class ConvolutionalNeuralNet(nn.Module):
    def __init__(self):
        super().__init__()
        self.relu = nn.ReLU()
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=6, kernel_size=5, stride=1, padding=2)
        self.pool1 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.conv2 = nn.Conv2d(in_channels=6, out_channels=16, kernel_size=5, stride=1, padding=0)
        self.pool2 = nn.AvgPool2d(kernel_size=2, stride=2)
        self.flatten = nn.Flatten()
        self.fc1 = nn.Linear(in_features=400, out_features=120)
        self.fc2 = nn.Linear(in_features=120, out_features=84)
        self.output = nn.Linear(in_features=84, out_features=10)

    def forward(self, x):
        x = self.relu(self.conv1(x))
        x = self.pool1(x)
        x = self.relu(self.conv2(x))
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.output(x)
        return x

In [30]:
model = ConvolutionalNeuralNet().to(device)

In [31]:
torchsummary.summary(model, (1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 6, 28, 28]             156
              ReLU-2            [-1, 6, 28, 28]               0
         AvgPool2d-3            [-1, 6, 14, 14]               0
            Conv2d-4           [-1, 16, 10, 10]           2,416
              ReLU-5           [-1, 16, 10, 10]               0
         AvgPool2d-6             [-1, 16, 5, 5]               0
           Flatten-7                  [-1, 400]               0
            Linear-8                  [-1, 120]          48,120
              ReLU-9                  [-1, 120]               0
           Linear-10                   [-1, 84]          10,164
             ReLU-11                   [-1, 84]               0
           Linear-12                   [-1, 10]             850
Total params: 61,706
Trainable params: 61,706
Non-trainable params: 0
---------------------------------

In [32]:
def get_accuracy(model, loader):
    model.eval()
    correct_predictions = 0
    with torch.no_grad():
        for X, y in loader:
            X, y = X.to(device), y.to(device)
            output = model(X)
            correct_predictions += torch.sum(torch.argmax(output, 1) == y).item()
    model.train()
    return correct_predictions / len(loader.dataset)

In [33]:
optimizer = torch.optim.AdamW(model.parameters(), lr=0.005, weight_decay=0)
lr_scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=2, gamma=0.5)
loss_fn = nn.CrossEntropyLoss()

epochs = 10
num_batches = len(train_loader)

model.train()
for i in range(epochs):
    epoch_loss = 0
    for X, y in train_loader:
        X, y = X.to(device), y.to(device)

        optimizer.zero_grad()

        output = model(X)
        loss = loss_fn(output, y)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    lr_scheduler.step()

    train_accuracy=get_accuracy(model, train_loader)
    dev_accuracy=get_accuracy(model, dev_loader)

    print(f"epoch: {i + 1}, loss = {epoch_loss/num_batches}, training accuracy = {train_accuracy}, dev_set accuracy = {dev_accuracy}")

epoch: 1, loss = 1.3538166783332826, training accuracy = 0.6594833333333333, dev_set accuracy = 0.8978
epoch: 2, loss = 0.8159477884451548, training accuracy = 0.73825, dev_set accuracy = 0.9546
epoch: 3, loss = 0.6757817324320475, training accuracy = 0.7727333333333334, dev_set accuracy = 0.957
epoch: 4, loss = 0.6383091763655344, training accuracy = 0.7912, dev_set accuracy = 0.9688
epoch: 5, loss = 0.5755110363960266, training accuracy = 0.80525, dev_set accuracy = 0.9728
epoch: 6, loss = 0.5613290061712265, training accuracy = 0.8043166666666667, dev_set accuracy = 0.9726
epoch: 7, loss = 0.5424824237942696, training accuracy = 0.8132666666666667, dev_set accuracy = 0.9708
epoch: 8, loss = 0.528786416554451, training accuracy = 0.8161333333333334, dev_set accuracy = 0.9706
epoch: 9, loss = 0.5193611969153087, training accuracy = 0.8229833333333333, dev_set accuracy = 0.9756
epoch: 10, loss = 0.5137050962924957, training accuracy = 0.82235, dev_set accuracy = 0.9764


In [34]:
import gradio as gr
from PIL import Image
import matplotlib.pyplot as plt

def predict(img):
    if isinstance(img, dict):
        img = img["composite"]

    if img is None:
        return {str(i): 0.0 for i in range(10)}


    pil = Image.fromarray(img)
    pil = pil.resize((28, 28)).convert('L')
    image = np.array(pil, dtype=np.float32) / 255.0

    image = 1.0 - image


    with torch.no_grad():
        model.eval()

        image = torch.from_numpy(image).unsqueeze(0).unsqueeze(0)
        # print(image.shape)
        output = model(image.to(device))
        prediction = torch.softmax(output, 1).cpu().numpy()[0]

        model.train()


    return {str(i): float(prediction[i]) for i in range(10)}

In [35]:
demo = gr.Interface(fn=predict, inputs=gr.Sketchpad(), outputs=gr.Label(num_top_classes=10), live=True)
demo.launch(share=True, debug=False)

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://1db3c3270c5d70cd50.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
